In [2]:
import numpy as np
import pandas as pd
from sklearn import *
from datetime import datetime
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

In [3]:
# read all the data
train = pd.read_csv('../train/training_data3.csv')
val = pd.read_csv('../train/validation_data3.csv')
val2 = pd.read_csv('../train/recent_validation.csv')

In [8]:
train = pd.concat([train, val], ignore_index=True)

In [9]:
col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors', 'dow', 'day_of_week', 'year', 
        'month', 'air_genre_name', 'air_area_name','weight']]
#col = [c for c in train if c not in ['id', 'air_store_id', 'visit_date','visitors', 'year','weight']]

In [10]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [11]:
train = train.fillna(-1)
val = val.fillna(-1)
val2 = val2.fillna(-1)

In [12]:
model1 = ensemble.GradientBoostingRegressor(learning_rate=0.13, random_state=3,
                    n_estimators=300, subsample=1.0, max_depth =6, min_weight_fraction_leaf=0.00006)
model1.fit(train[col], np.log1p(train['visitors'].values), sample_weight=train['weight'])
preds1 = model1.predict(train[col])
print('Training - RMSE GradientBoostingRegressor: ', RMSLE(np.log1p(train['visitors'].values), preds1))
preds1 = model1.predict(val[col])
print('Val - RMSE GradientBoostingRegressor: ', RMSLE(np.log1p(val['visitors'].values), preds1))
preds1 = model1.predict(val2[col])
print('Val2 - RMSE GradientBoostingRegressor: ', RMSLE(np.log1p(val2['visitors'].values), preds1))

Training - RMSE GradientBoostingRegressor:  0.4769874938861581
Val - RMSE GradientBoostingRegressor:  0.47680088811722854
Val2 - RMSE GradientBoostingRegressor:  0.4040007990094951


In [13]:
preds1 = model1.predict(train[col])
train['predict'] = np.expm1(preds1)

In [14]:
train.to_csv("output.csv", index=False)

In [8]:
model3 = XGBRegressor(learning_rate=0.13, random_state=3, n_estimators=300, subsample=1.0, 
                      colsample_bytree=0.8, max_depth =6)
model3.fit(train[col], np.log1p(train['visitors']), sample_weight = train['weight'])
preds3 = model3.predict(train[col])
print('Train error ', RMSLE(np.log1p(train['visitors'].values), preds3))
preds3 = model3.predict(val[col])
print('Val error ', RMSLE(np.log1p(val['visitors'].values), preds3))
preds3 = model3.predict(val2[col])
print('Val2 error ', RMSLE(np.log1p(val2['visitors'].values), preds3))

Train error  0.47875875692334563
Val error  0.5051103327012156
Val2 error  0.47472231433534007


In [16]:
val2['predict'] = np.expm1(preds3)

In [41]:
tmp = val2[['air_store_id', 'visit_date','day_of_week',
       'holiday_flg', 'visitors', 'weighted_visits', 'predict','mean_visitors', 'median_visitors', 'lastm_min_visitors',
       'lastm_mean_visitors', 'lastm_median_visitors', 'lastm_max_visitors',
       'last_twom_min_visitors', 'last_twom_mean_visitors',
       'last_twom_median_visitors', 'last_twom_max_visitors' ]]

In [42]:
tmp = tmp.sort_values(['air_store_id', 'visit_date'], ascending=[True, True])

In [43]:
tmp.sort_values(['air_store_id', 'visit_date'], ascending=[True, True])

,air_store_id,visit_date,day_of_week,holiday_flg,visitors,weighted_visits,predict,mean_visitors,median_visitors,lastm_min_visitors,lastm_mean_visitors,lastm_median_visitors,lastm_max_visitors,last_twom_min_visitors,last_twom_mean_visitors,last_twom_median_visitors,last_twom_max_visitors
823,air_00a91d42b08b08d9,2017-04-01,Saturday,0,7,11.692007,9.917704,14.973684,11.0,4.0,9.500000,8.0,18.0,4.0,12.857143,11.0,23.0
869,air_00a91d42b08b08d9,2017-04-04,Tuesday,0,17,25.822879,20.712622,24.350000,24.5,28.0,29.250000,29.0,31.0,19.0,30.625000,30.5,41.0
3669,air_00a91d42b08b08d9,2017-04-08,Saturday,0,9,11.136156,9.540903,14.973684,11.0,4.0,9.500000,8.0,18.0,4.0,12.857143,11.0,23.0
1861,air_00a91d42b08b08d9,2017-04-12,Wednesday,0,28,28.999368,29.198242,28.125000,28.0,17.0,32.400000,28.0,52.0,15.0,32.111111,28.0,52.0
1099,air_00a91d42b08b08d9,2017-04-17,Monday,0,19,24.199201,19.443806,22.457143,19.0,18.0,33.333333,39.0,43.0,15.0,30.000000,34.0,43.0
185,air_00a91d42b08b08d9,2017-04-22,Saturday,0,18,10.915509,11.320033,14.973684,11.0,4.0,9.500000,8.0,18.0,4.0,12.857143,11.0,23.0
2438,air_0164b9927d20bcc3,2017-04-03,Monday,0,13,6.040526,14.072042,7.500000,6.0,4.0,8.000000,9.0,11.0,2.0,5.833333,4.5,11.0
136,air_0164b9927d20bcc3,2017-04-05,Wednesday,0,20,9.006755,11.288357,9.678571,8.0,6.0,13.200000,11.0,24.0,2.0,10.555556,8.0,24.0
1582,air_0164b9927d20bcc3,2017-04-08,Saturday,0,1,4.235314,4.517526,6.409091,4.5,1.0,4.250000,3.5,9.0,1.0,4.125000,2.0,11.0
2734,air_0164b9927d20bcc3,2017-04-13,Thursday,0,13,9.251708,12.532132,9.846154,10.5,5.0,11.250000,12.0,16.0,5.0,11.714286,13.0,16.0


In [9]:
# Get the final output
test = pd.read_csv('../train/filled_test.csv')
#test = test.fillna(-1)
preds1 = model1.predict(test[col])
test['visitors'] = preds1
test['visitors'] = np.expm1(test['visitors']).clip(lower=0.)
result = test[['id', 'visitors']]
result.to_csv('result_0202_01.csv',index=False)

In [ ]:
params = {'n_estimators' : 300}
# Plot training deviance
# compute test set deviance
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)

for i, y_pred in enumerate(model1.staged_predict(val[col])):
    test_score[i] = model1.loss_(np.log1p(val['visitors'].values), y_pred)

plt.figure(figsize=(12, 6))
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, model1.train_score_, 'b-', label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-', label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')
plt.show()

In [ ]:
# Plot feature importance
col_array = np.asarray(col)
feature_importance = model1.feature_importances_
# make importances relative to max importance
feature_importance = 100.0 * (feature_importance / feature_importance.max())
sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
plt.figure(figsize=(12, 20))
plt.barh(pos, feature_importance[sorted_idx], align='center')
plt.yticks(pos, col_array[sorted_idx])
plt.xlabel('Relative Importance')
plt.title('Variable Importance')
plt.show()